# Data Science Lab: Lab 5 - Anika Singh, Ravi Akalkotkar, Siddhant Singh
**Due: Monday, March 7, 11:59 pm**

Submit:
1. A pdf of your notebook with solutions.
2. A link to your colab notebook or also upload your .ipynb if not working on colab.

# Goals of this Lab

1. Understanding Entropy
2. Scraping data.
3. Intro to Logistic Regression
4. Revisiting CIFAR-10 and MNIST.

## Problem 1 (Optional)

Read Shannon's 1948 paper 'A Mathematical Theory of Communication'. Focus on pages 1-19 (up to Part II), the remaining part is more relevant for communication. 

https://math.harvard.edu/~ctm/home/text/others/shannon/entropy/entropy.pdf


## Problem 2: Scraping, Entropy and ICML papers

ICML -- the International Conference on Machine Learning -- is a top research conference in Machine learning. Scrape all the pdfs of all ICML 2021 papers from http://proceedings.mlr.press/v139/, and answer the following questions.

1. What are the top 10 common words in the ICML papers?  
2.  Let $Z$ be a randomly selected word in a randomly selected ICML paper. Estimate the entropy of $Z$.
3.  Synthesize a random paragraph using the marginal distribution over words. 
4. (Optional) Synthesize a random paragraph using an n-gram model on words. Synthesize a random paragraph using any model you want. We will learn more about generating text in later classes when we get into NLP. 

Note: downloading may take some time. 

In [1]:
!pip install PyPdf2
!pip install pdfminer

In [3]:
import requests
import io
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfparser import PDFParser
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import numpy as np
import os
from scipy.stats import entropy
from sklearn.metrics import log_loss
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer
from pdfminer.pdfpage import PDFPage
from io import StringIO
from pdfminer.converter import TextConverter
from pdfminer.high_level import extract_text
from PyPDF2 import PdfFileReader


In [4]:
url = "http://proceedings.mlr.press/v139/"

read = requests.get(url)
html_content = read.content
soup = BeautifulSoup(html_content, "html.parser")
list_of_pdf = list()
l = soup.find_all("div", {"class": "paper"})
 
count = 0
# iterate through p for getting all the href links
for link in l:
    # accessed all the anchors tag from given p tag
    p = link.find_all('a')
    for a in p[1::3]:
      count += 1
      # converting the extension from .html to .pdf
      pdf_link = (a.get('href')[:-4]) + ".pdf"
      
      # added all the pdf links to set
      list_of_pdf.append(pdf_link)

print(count)

1183


In [9]:
for i in range(0,400): #scraping the first 400 (1/3rd of the entire data set), since Colab has insufficent RAM and I want to be able to complete the lab, so used smaller data set
  response = requests.get(list_of_pdf[i])
  text = extract_text(io.BytesIO(response.content))
  f = open("myfile.txt", "a")
  f.write(text)
  f.close()

#f = open("myfile.txt", "r")
#print(f.read())



**1. What are the top 10 common words in the ICML papers?** (FIRST 400 PDFs ONLY aka 1/3rd of the entire data set)

In [10]:
with open ("myfile.txt", "r", encoding = "utf-8") as myfile:
  textOne = myfile.readlines()

vectorize = CountVectorizer()
vectorize.fit(textOne)
listOfWordsandOccurences = vectorize.vocabulary_

with open('finalCountOfWords.txt', 'w', encoding = "utf-8") as fileOne:
  print(vectorize.vocabulary_, file = fileOne)  

NewVectorize = CountVectorizer().fit(textOne)
words = NewVectorize.transform(textOne)

wordNum = words.sum(axis = 0)

frequencyOfWord = [(word, wordNum[0,idx]) for word, idx in NewVectorize.vocabulary_.items()]
frequencyOfWord =sorted(frequencyOfWord, key = lambda x: x[1], reverse=True)
top10 = frequencyOfWord[:10]

print("Top 10 words in first 400 PDFs: \n", top10)



Top 10 words in first 400 PDFs: 
 [('the', 190764), ('of', 97806), ('and', 91622), ('in', 74814), ('to', 64075), ('cid', 54129), ('for', 48315), ('is', 48040), ('we', 44840), ('that', 33330)]


**2. Let  Z  be a randomly selected word in a randomly selected ICML paper. Estimate the entropy of  Z .**

In [8]:
dictionaryProb = dict()
for x,y in frequencyOfWord:
  dictionaryProb.setdefault(a,[]).append(y)

listOfProbabailites = list(dictionaryProb.values())
#convert to pandas ds
pd_series = pd.Series(listOfProbabailites)
counts = pd_series.value_counts()
entropyOne = entropy(counts)

print("Entropy is: ", entropyOne)

Entropy is:  8.652721131567146


## Problem 3: Logistic Regression

The following is a logistic regression problem using a real data set, made available by the authors of the book ``Applied Regression and Muiltilevel Modeling'' by Gelman and Hill. 

Download the data from the book, which you can find here http://www.stat.columbia.edu/~gelman/arm/software/. In particular, we are interested in the **arsenic** data set. The file **wells.dat** contains data on 3,020 households in Bangladesh. For each family, the natural arsenic level of each well was measured. In addition, the distance to the nearest safest well was measured. Each family is also described by a feature that relates to their community involvement, and a feature that gives the education level of the head of household. We are interested in building a model that predicts whether the family decided to switch wells or not, based on being informed of the level of arsenic in the well. Thus the "label" for this problem is the binary vector that is the first column of the dataset, labeled "switch."

1. Fit a logistic regression model using only an offset term and the distance to the nearest safe well.
2. Plot your answer: that is, plot the probability of switching wells as a function of the distance to the nearest safe well. 
3. Interpreting logistic regression coefficients: Use the "rule-of-4" discussed in class on Thursday, to interpret the solution: what can you say about the change in the probability of switching wells, for every additional 100 meters of distance?
4. Now solve a logistic regression incorporating the constant term, the distance and also arsenic levels. Report the coefficients
5. Next we want to answer the question of which factor is more significant, distance, or arsenic levels? This is not a well specified question, since these two features have different units. One natural choice is to ask if after normalizing by the respective standard deviations of each feature, if moving one unit in one (normalized) feature predicts a larger change in probability of switching wells, than moving one unit in the other (also normalized) feature. Use this reasoning to answer the question. 
6. Now consider all the features in the data set. Also consider adding interaction terms among all features that have a large main effect. Use cross validation to build the best model you can (using your training set only), and then report the test error of your best model. (Note that since you have essentially unlimited access to your test set, this opens the door for massive overfitting. In contrast, Kaggle competitions try to mollify this by giving you only limited access to the test set.)
7. (Optional) Now also play around with $\ell_1$ and $\ell_2$ regularization, and try to build the most accurate model you can (accuracy computed on the test data). 

In [ ]:
# convert dat file to pandas dataframe
df = pd.read_csv('arsenic/wells.dat', sep = ' ', header = 0, index_col = 0)
print(df.head())

1. Fit a logistic regression model using only an offset term and the distance to the nearest safe well

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

X = np.column_stack((np.ones(df.shape[0]).T, df['dist']))
y = df['switch']

log = LogisticRegressionCV(Cs = [0.01, 0.1, 1, 10, 100], cv=5)
log_fit = log.fit(X, y)

2. Plot your answer: that is, plot the probability of switching wells as a function of the distance to the nearest safe well.

In [ ]:
len_dist = max(df['dist'].values)
dist_count = np.linspace(0, int(len_dist)-1, int(len_dist))

distance_range = np.column_stack((np.ones(int(len_dist)), dist_count))

prob_switching = log_fit.predict_proba(distance_range)

plt.plot(dist_count, prob_switching[:, 0]) 
plt.xlabel("Distance to Nearest Safe Well")
plt.ylabel("Probability of Switching Wells Predictions")
plt.show()

3. Interpreting logistic regression coefficients: Use the "rule-of-4" discussed in class on Thursday, to interpret the solution: what can you say about the change in the probability of switching wells, for every additional 100 meters of distance? **every additional 100m results in 15% increase chance of switching wells**

In [ ]:
print(log_fit.coef_)
print(log_fit.intercept_)

4. Now solve a logistic regression incorporating the constant term, the distance and also arsenic levels. Report the coefficients

In [ ]:
X_normalize = np.column_stack((df['dist'], df['arsenic']))
X_normalize = (X_normalize - np.mean(X_normalize))/np.std(X_normalize)

print(X_normalize.shape, df.shape[0])

X_2 = np.column_stack((np.ones(df.shape[0]).T, X_normalize))
y_2 = df['switch']

log_2 = LogisticRegressionCV(Cs = [0.01, 0.1, 1, 10, 100], cv=5)
log_fit_2 = log_2.fit(X_2, y_2)

print(log_fit_2.coef_)

5. Next we want to answer the question of which factor is more significant, distance, or arsenic levels? This is not a well specified question, since these two features have different units. One natural choice is to ask if after normalizing by the respective standard deviations of each feature, if moving one unit in one (normalized) feature predicts a larger change in probability of switching wells, than moving one unit in the other (also normalized) feature. Use this reasoning to answer the question.

**According to the coefficients above, the most important feature is the arsenic levels**

6. Now consider all the features in the data set. Also consider adding interaction terms among all features that have a large main effect. Use cross validation to build the best model you can (using your training set only), and then report the test error of your best model. (Note that since you have essentially unlimited access to your test set, this opens the door for massive overfitting. In contrast, Kaggle competitions try to mollify this by giving you only limited access to the test set.)

In [ ]:
X_3 = np.stack((np.ones(df.shape[0]).T, df['dist'], df['arsenic'], df['assoc'], df['educ'])).T

fit_log_3 = LogisticRegressionCV(Cs=np.logspace(-3, 3, num=10), cv=10).fit(X_3, y)

print(fit_log_3.score(X_3, y))

## Problem 4: Logistic Regression and CIFAR-10

In this problem you will explore the data set CIFAR-10, and you will use multinomial (multi-label) Logistic Regression to try to classify it. You will also explore visualizing the solution.

1. (Optional) You can read about the CIFAR-10 and CIFAR-100 data sets here: https://www.cs.toronto.edu/~kriz/cifar.html.
2. (Optional) OpenML curates a number of data sets. You will use a subset of CIFAR-10 provided by them. Read here for a description: https://www.openml.org/d/40926.
3. Use the **fetch_openml** command from **sklearn.datasets** to import the CIFAR-10-Small data set. There are 20,000 data points. Do a train-test split on 3/4 - 1/4.
4. Figure out how to display some of the images in this data set, and display a couple. While not high resolution, these should be recognizable if you are doing it correctly.
5. You will run multi-class logistic regression on these using the cross entropy loss. You have to specify this specifically (**multi_class='multinomial'**). Use cross validation to see how good your accuracy can be. In this case, cross validate to find as good regularization coefficients as you can, for $\ell_1$ and $\ell_2$ regularization (i.e., for the Lasso and Ridge penalties), which are naturally supported in **sklearn.linear_model.LogisticRegression**. I recommend you use the solver **saga**. Note that this is quite a large problem: $20,000$ data points, each of them $3,072$-dimensional. Report your training and test losses.
6. How sparse can you make your solutions without deteriorating your testing error too much? Here, I am asking you to try to obtain a sparse solution that has test accuracy that is close to the best solution you found. 


In [ ]:
from sklearn.datasets import fetch_openml
cifar_small = fetch_openml('CIFAR_10_Small', version=1) 

Use the fetch_openml command from sklearn.datasets to import the CIFAR-10-Small data set. There are 20,000 data points. Do a train-test split on 3/4 - 1/4.

In [ ]:
from sklearn.model_selection import train_test_split

X, y = cifar_small['data'], cifar_small['target']
print(X.shape)

# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
print(X_train.shape)

Figure out how to display some of the images in this data set, and display a couple. While not high resolution, these should be recognizable if you are doing it correctly.

In [ ]:
for i in range(4):
    img = X.iloc[i].to_numpy()
    image = np.dstack((img[0:1024].reshape(32,32)/255.0,img[1024:2048].reshape(32,32)/255.0,img[2048:].reshape(32,32)/255.0))
    plt.figure()
    plt.imshow(image)

You will run multi-class logistic regression on these using the cross entropy loss. You have to specify this specifically (multi_class='multinomial'). Use cross validation to see how good your accuracy can be. In this case, cross validate to find as good regularization coefficients as you can, for  ℓ1  and  ℓ2  regularization (i.e., for the Lasso and Ridge penalties), which are naturally supported in sklearn.linear_model.LogisticRegression. I recommend you use the solver saga. Note that this is quite a large problem:  20,000  data points, each of them  3,072 -dimensional. Report your training and test losses.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

logistic = LogisticRegression(multi_class='multinomial')
C = np.logspace(-2, 2, 5)

cv = 5
param_grid = [{'penalty': ['l1'], 'solver':['saga'], 'C': C}, {'penalty': ['l2'], 'solver':['saga'], 'C': C}]

log_reg_search = GridSearchCV(logistic, param_grid, cv=cv)
log_reg_search.fit(X_train, y_train)

log_reg_search.best_params_

In [ ]:
best_est = log_reg_search.best_estimator_
train_pred = best_est.predict_proba(X_train)
print('training loss: ', log_loss(y_train, train_pred))

test_pred = best_est.predict_proba(X_test)
print('testing loss: ', log_loss(y_test, test_pred))

In [ ]:
print(log_reg_search.best_estimator_)
score = log_reg_search.best_estimator_.score(X_test, y_test)
print('Test Accuracy Score: ', score)

How sparse can you make your solutions without deteriorating your testing error too much? Here, I am asking you to try to obtain a sparse solution that has test accuracy that is close to the best solution you found.

In [ ]:
from sklearn.linear_model import LogisticRegression

sparse_penalty = [0.1, 1, 10, 100]

for i in sparse_penalty:
    logistic = LogisticRegression(multi_class='multinomial', penalty = 'l1', solver = 'saga', C = i)
    logistic.fit(X_train, y_train)
    
    train_pred = logistic.predict_proba(X_train)
    print('C: ', i)
    print('training loss: ', log_loss(y_train, train_pred))

    test_pred = logistic.predict_proba(X_test)
    print('testing loss: ', log_loss(y_test, test_pred))
    
    score = logistic.score(X_test, y_test)
    print('Test Accuracy Score: ', score)
    

In [ ]:
from sklearn.linear_model import LogisticRegression

sparse_penalty = [1000, 10000]

for i in sparse_penalty:
    logistic = LogisticRegression(multi_class='multinomial', penalty = 'l1', solver = 'saga', C = i)
    logistic.fit(X_train, y_train)
    
    train_pred = logistic.predict_proba(X_train)
    print('C: ', i)
    print('training loss: ', log_loss(y_train, train_pred))

    test_pred = logistic.predict_proba(X_test)
    print('testing loss: ', log_loss(y_test, test_pred))
    
    score = logistic.score(X_test, y_test)
    print('Test Accuracy Score: ', score)
    

In [ ]:
from sklearn.linear_model import LogisticRegression

sparse_penalty = [0.001]

for i in sparse_penalty:
    logistic = LogisticRegression(multi_class='multinomial', penalty = 'l1', solver = 'saga', C = i)
    logistic.fit(X_train, y_train)
    
    train_pred = logistic.predict_proba(X_train)
    print('C: ', i)
    print('training loss: ', log_loss(y_train, train_pred))

    test_pred = logistic.predict_proba(X_test)
    print('testing loss: ', log_loss(y_test, test_pred))
    
    score = logistic.score(X_test, y_test)
    print('Test Accuracy Score: ', score)

In [ ]:
from sklearn.linear_model import LogisticRegression

sparse_penalty = [0.0001]

for i in sparse_penalty:
    logistic = LogisticRegression(multi_class='multinomial', penalty = 'l1', solver = 'saga', C = i)
    logistic.fit(X_train, y_train)
    
    train_pred = logistic.predict_proba(X_train)
    print('C: ', i)
    print('training loss: ', log_loss(y_train, train_pred))

    test_pred = logistic.predict_proba(X_test)
    print('testing loss: ', log_loss(y_test, test_pred))
    
    score = logistic.score(X_test, y_test)
    print('Test Accuracy Score: ', score)

## Problem 5: Multi-class Logistic Regression -- Visualizing the Solution

You will repeat the previous problem but for the MNIST data set which you will find here: https://www.openml.org/d/554. As we have seen before, MNIST is a data set of handwritten digits, and is considered one of the "easiest" image recognition problems in computer vision. We will see here how well logistic regression does, as you did above on the CIFAR-10 subset. In addition, we will see that we can visualize the solution, and that in connection to this, sparsity can be useful.


1. Use the **fetch_openml** command from to import the MNIST data set, and choose a reasonable train-test split (e.g., 75-25).
2. Again run multi-class logistic regression on these using the cross entropy loss, as you did above. Try to optimize the hyperparameters. Report your training and test loss.
3. Choose an $\ell_1$ regularizer (penalty), and see if you can get a sparse solution with almost as good accuracy.
4. Note that in Logistic Regression, the coefficients returned (i.e., the $\beta$'s) are the same dimension as the data. Therefore we can pretend that the coefficients of the solution are an image of the same dimension, and plot it. Do this for the 10 sets of coefficients that correspond to the 10 classes. You should observe that, at least for the sparse solutions, these "kind of" look like the digits they are classifying. 


In [ ]:
import time
import matplotlib.pyplot as plt
import numpy as np
from sklearn import tree
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_random_state
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss

X, y = fetch_openml('mnist_784', version=1, return_X_y=True)

X = X.to_numpy()
y = y.to_numpy()

X.shape
X_train = X[:52500]
y_train = y[:52500]
X_test = X[52500:]
y_test = y[52500:]

In [ ]:
# using the code from one of the lecture colabs to display a digit just to make sure it works
index = 12
img = X[index]
label = y[index]
img = img.reshape((28, 28))

plt.title('The label is {label}'.format(label=label))
plt.imshow(img, cmap='gray')
plt.show()

2. Again run multi-class logistic regression on these using the cross entropy loss, as you did above. Try to optimize the hyperparameters. Report your training and test loss.

In [ ]:
logistic = LogisticRegression(multi_class='multinomial', tol = 0.1, solver='saga')
C = np.logspace(-2, 2, 5)

param_grid = [{'penalty': ['l1'], 'C': C}, {'penalty': ['l2'], 'C': C}]

log_reg_search = GridSearchCV(logistic, param_grid)
log_reg_search.fit(X_train, y_train)

log_reg_search.best_params_

In [ ]:

logistic = LogisticRegression(multi_class='multinomial', penalty = 'l2', tol=0.1, solver = 'saga', C = 0.1)
logistic.fit(X_train, y_train)

train_pred = logistic.predict_proba(X_train)
print('training loss: ', log_loss(y_train, train_pred))
test_pred = logistic.predict_proba(X_test)
print('testing loss: ', log_loss(y_test, test_pred))

score = logistic.score(X_test, y_test)
print('Test Accuracy Score: ', score)

In [ ]:
C = np.linspace(0.1, 10, num=100) #searched over this array earlier, took forever but got 6.7

logistic = LogisticRegression(multi_class='multinomial', tol = 0.1, solver= 'saga', C=6.7, penalty='l2');
logistic.fit(X_train, y_train)

train_pred = logistic.predict_proba(X_train)
print('training loss: ', log_loss(y_train, train_pred))
test_pred = logistic.predict_proba(X_test)
print('testing loss: ', log_loss(y_test, test_pred))

score = logistic.score(X_test, y_test)
print('Test Accuracy Score: ', score)

3. Choose an  ℓ1  regularizer (penalty), and see if you can get a sparse solution with almost as good accuracy.

In [ ]:
logistic = LogisticRegression(multi_class='multinomial', tol = 0.1, penalty = 'l1', solver = 'saga')
C = np.logspace(-2, 2, 5)

param_grid = [{'C': C}]

log_reg_search = GridSearchCV(logistic, param_grid)
log_reg_search.fit(X_train, y_train)

log_reg_search.best_params_

In [ ]:
logistic = LogisticRegression(multi_class='multinomial', tol = 0.1, penalty = 'l1', solver = 'saga', C=10)
logistic.fit(X_train, y_train)

train_pred = logistic.predict_proba(X_train)
print('training loss: ', log_loss(y_train, train_pred))
test_pred = logistic.predict_proba(X_test)
print('testing loss: ', log_loss(y_test, test_pred))

score = logistic.score(X_test, y_test)
print('Test Accuracy Score: ', score)

In [ ]:
logistic = LogisticRegression(multi_class='multinomial', tol = 0.1, penalty = 'l1', solver = 'saga')
C = np.logspace(-5, -1, 10)
param_grid = [{'C': C}]
log_reg_search = GridSearchCV(logistic, param_grid)
log_reg_search.fit(X_train, y_train)
log_reg_search.best_params_

In [ ]:
logistic = LogisticRegression(multi_class='multinomial', tol = 0.1, penalty = 'l1', solver = 'saga', C=0.0359)
logistic.fit(X_train, y_train)

train_pred = logistic.predict_proba(X_train)
print('training loss: ', log_loss(y_train, train_pred))
test_pred = logistic.predict_proba(X_test)
print('testing loss: ', log_loss(y_test, test_pred))

score = logistic.score(X_test, y_test)
print('Test Accuracy Score: ', score)

4. Note that in Logistic Regression, the coefficients returned (i.e., the  𝛽 's) are the same dimension as the data. Therefore we can pretend that the coefficients of the solution are an image of the same dimension, and plot it. Do this for the 10 sets of coefficients that correspond to the 10 classes. You should observe that, at least for the sparse solutions, these "kind of" look like the digits they are classifying.

In [ ]:

for index in range(0, 10):
    img = logistic.coef_[index]
    img = img.reshape((28, 28))
    plt.title('The label is {index}'.format(index=index))
    plt.imshow(img, cmap='gray')
    plt.show()